# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [3]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint
# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [4]:
#Saves random cities csv data to new data frame
gmap_df = pd.read_csv('random_cities.csv')
gmap_df

,Unnamed: 0,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,4032243,Vaini,75,TO,1612492408,89,-21.2000,-175.2000,80.60,5.75
1,1,3833367,Ushuaia,40,AR,1612492408,81,-54.8000,-68.3000,44.60,34.52
2,2,674735,Lipniţa,2,RO,1612492408,91,44.1000,27.6000,45.00,1.01
3,3,1510689,Baykit,100,RU,1612492408,88,61.6700,96.3700,-0.63,3.13
4,4,4031637,Lavrentiya,100,RU,1612492408,89,65.5833,-171.0000,-7.60,17.92
...,...,...,...,...,...,...,...,...,...,...,...
561,561,2277060,Gbarnga,51,LR,1612492480,94,6.9956,-9.4722,74.01,1.95
562,562,5105168,Sussex,1,US,1612492480,80,41.1834,-74.6663,28.99,2.28
563,563,3472520,Almenara,33,BR,1612492481,90,-16.1836,-40.6944,71.65,1.83
564,564,1563281,Tuy Hoa,36,VN,1612492378,81,13.0833,109.3000,75.52,11.72


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [24]:
gmaps.configure(api_key=g_key) #Load API key

#Establish Lat and Lng columns as coordinates
locations = gmap_df[['Lat','Lng']]
#Establish Humidity column with float values
humidity = gmap_df['Humidity'].astype(float)

#Set figure layout
figure_layout = {
    'width': '500px',
    'height': '500px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
#Applies figure layout to figure
fig = gmaps.figure(layout=figure_layout)
#Creates heat layer and adds it to figure
heat_layer = gmaps.heatmap_layer(locations,weights=humidity,dissipating=True,
                                 max_intensity=40,point_radius=7)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='500px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [25]:
#A max temperature lower than 80 degrees but higher than 70.
#Wind speed less than 10 mph.
#Zero cloudiness.
gmap_df_2 = gmap_df.drop(gmap_df[(gmap_df['Max Temp']<70) | (gmap_df['Max Temp']>80)].index)
gmap_df_2 = gmap_df_2.drop(gmap_df_2[(gmap_df_2['Wind Speed'] >= 10)].index)
gmap_df_2 = gmap_df_2.drop(gmap_df_2[(gmap_df['Cloudiness'] != 0)].index)
#for i, city in gmap_df_2.iterrows():
#    try:
#        if (city[i,'Max Temp'] < 70 | city[i,'Max Temp'] > 80) & (city[i,'Wind Speed'] >= 10) & (city[i,'Cloudiness'] != 0):
#            city = []
#    except:
#        print('Data not available')
gmap_df_2

<ipython-input-25-53c02069cda3>:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  gmap_df_2 = gmap_df_2.drop(gmap_df_2[(gmap_df['Cloudiness'] != 0)].index)


,Unnamed: 0,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
78,78,2336056,Jimeta,0,NG,1612492417,24,9.2833,12.4667,70.02,2.17
89,89,3837213,San Juan,0,AR,1612492418,22,-31.5375,-68.5364,78.31,6.29
123,123,2294915,Takoradze,0,GH,1612492422,88,4.8845,-1.7554,78.85,7.65
161,161,2318921,Wukari,0,NG,1612492427,19,7.8500,9.7833,75.56,6.44
170,170,3385077,Conde,0,BR,1612492428,83,-7.2597,-34.9075,78.80,5.75
205,205,934479,Grand Gaube,0,MU,1612492432,76,-20.0064,57.6608,78.01,5.01
275,275,3470137,Belmonte,0,BR,1612492440,78,-15.8631,-38.8828,78.80,5.75
307,307,3466980,Caravelas,0,BR,1612492445,83,-17.7125,-39.2481,77.14,8.10
308,308,374739,Gogrial,0,SS,1612492445,22,8.5332,28.1004,72.07,8.63
367,367,3866163,Alta Gracia,0,AR,1612492165,83,-31.6529,-64.4283,70.00,1.99


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [26]:
#Create hotel_df and create new column Hotel Name
hotel_df = gmap_df_2
hotel_df['Hotel Name'] = ''

#Base url and preliminary parameters for API call
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {'type':'lodging','radius':5000,'key':g_key}

#Establishes for loop for lat/lng conversion and API calls
for i,row in hotel_df.iterrows():
    #Store Lat and Lng and insert into new dictionary key
    #called 'location'
    lat = row['Lat']
    lng = row['Lng']
    params['location'] = f'{lat},{lng}'
    #API request
    response = requests.get(base_url,params=params).json()
    try:
        #Tries to store first hotel name from response in
        #Hotel Name column
        hotel_df.loc[i, 'Hotel Name'] = response['results'][0]['name']
    except:
        #Puts 'No hotel found' if hotel not found
        hotel_df.loc[i, 'Hotel Name'] = 'No hotel found'
hotel_df

,Unnamed: 0,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
78,78,2336056,Jimeta,0,NG,1612492417,24,9.2833,12.4667,70.02,2.17,Yola International Hotel
89,89,3837213,San Juan,0,AR,1612492418,22,-31.5375,-68.5364,78.31,6.29,Hotel Provincial
123,123,2294915,Takoradze,0,GH,1612492422,88,4.8845,-1.7554,78.85,7.65,Raybow International Hotel
161,161,2318921,Wukari,0,NG,1612492427,19,7.8500,9.7833,75.56,6.44,Elimtop
170,170,3385077,Conde,0,BR,1612492428,83,-7.2597,-34.9075,78.80,5.75,Pousada Beija Flor
205,205,934479,Grand Gaube,0,MU,1612492432,76,-20.0064,57.6608,78.01,5.01,Veranda Paul et Virginie Hotel & Spa
275,275,3470137,Belmonte,0,BR,1612492440,78,-15.8631,-38.8828,78.80,5.75,Pousada Monte Carmelo O Pão
307,307,3466980,Caravelas,0,BR,1612492445,83,-17.7125,-39.2481,77.14,8.10,Pousada dos Navegantes
308,308,374739,Gogrial,0,SS,1612492445,22,8.5332,28.1004,72.07,8.63,No hotel found
367,367,3866163,Alta Gracia,0,AR,1612492165,83,-31.6529,-64.4283,70.00,1.99,Hotel Ritz


In [27]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [28]:
# Add marker layer ontop of heat map
marker = gmaps.marker_layer(locations,info_box_content=hotel_info)
fig.add_layer(marker)
# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='500px', margin='0 auto 0 auto', padding='1px', wi…